# **Part 1**

## First, I need to import something

In [0]:
import pandas as pd
import numpy as np

## Read CSV file 
I downloaded it and import to this path: '/content/PostCode_Canada.csv'

In [427]:
df = pd.read_csv('/content/PostCode_Canada.csv')
df.head()

,Postcode,Borough,Neighbourhood
0,M9Z,Not assigned,Not assigned
1,M9Y,Not assigned,Not assigned
2,M9X,Not assigned,Not assigned
3,M9W,Etobicoke,Northwest
4,M9V,Etobicoke,Albion Gardens


##Remove *Not assigned* value in **Borough**


In [428]:
df["Borough"].replace("Not assigned", np.nan, inplace=True)
df.dropna(subset=["Borough"], axis=0, inplace=True)
df.index = range(len(df.index))
df.head()

,Postcode,Borough,Neighbourhood
0,M9W,Etobicoke,Northwest
1,M9V,Etobicoke,Albion Gardens
2,M9V,Etobicoke,Beaumond Heights
3,M9V,Etobicoke,Humbergate
4,M9V,Etobicoke,Jamestown


## Next, *Not assigned* in **Neighbourhood** is filled by value of **Borough**

In [0]:
for i, row in df.iterrows():
  if df.loc[i]["Neighbourhood"] == "Not assigned":
    df.loc[i]["Neighbourhood"] = df.loc[i]["Borough"]

##Combinate all same Postcode values.

In [0]:
pre_i = 0
for i, row in df.iterrows():
  if i == 0:
    continue
  if df.loc[pre_i]["Postcode"] == row["Postcode"]:
    df.loc[pre_i]["Neighbourhood"] += ", " + row["Neighbourhood"]
    df.drop(labels=i, axis=0, inplace=True)
  else:
    pre_i = i

## Finally, reindex df and show it


In [431]:
df.index = range(len(df.index))
df.head()

,Postcode,Borough,Neighbourhood
0,M9W,Etobicoke,Northwest
1,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
2,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
3,M9P,Etobicoke,Westmount
4,M9N,York,Weston


In [432]:
df.shape

(103, 3)

#**Part 2**
Because it is too long for waiting response from geocoder. So I use Geospatial_Coordinated.csv file.

In [433]:
geo_df = pd.read_csv('/content/Geospatial_Coordinates.csv')
geo_df.rename(columns={"Postal Code" : "Postcode"}, inplace=True)
geo_df.set_index(keys="Postcode", inplace=True)
geo_df.head()

,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [434]:
df["Latitude"] = np.zeros(df.shape[0])
df["Longitude"] = np.zeros(df.shape[0])
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9W,Etobicoke,Northwest,0.0,0.0
1,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0
2,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",0.0,0.0
3,M9P,Etobicoke,Westmount,0.0,0.0
4,M9N,York,Weston,0.0,0.0


In [0]:
for i, row in df.iterrows():
  postcode = row["Postcode"]
  df.loc[i, "Latitude"] = geo_df.loc[postcode, "Latitude"]
  df.loc[i, "Longitude"] = geo_df.loc[postcode, "Longitude"]

And this is our result

In [436]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9W,Etobicoke,Northwest,43.706748,-79.594054
1,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
2,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,M9N,York,Weston,43.706876,-79.518188
